# PAA -  Greed 



 O primeiro passo é definir a estrutura da nossa heap, que será utilizada para guardar cada um dos nosso caracteres em conjunto com sua frequência e nós a esquerda e a direita. 

In [1]:
import heapq
import os
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

Agora podemos pedir ao usuário o caminho do arquivo a ser comprimido. Em seguida preenchemos o nosso heap com esses dados

In [7]:
original_path = input("Digite o caminho do arquivo:")
original_filename, original_file_extension = os.path.splitext(original_path)
with open(original_path, 'r+') as file:
    text = file.read()
    text = text.rstrip()

chars_frequency = {}
for character in text:
    if not character in chars_frequency:
        chars_frequency[character] = 0
    chars_frequency[character] += 1

print(chars_frequency)

heap = []

for key in chars_frequency:
    node = HeapNode(key, chars_frequency[key])
    heapq.heappush(heap, node)

Digite o caminho do arquivo:lorem.txt
{'Q': 12, 'u': 49, 'a': 111, 'n': 39, 'd': 36, 'o': 108, ' ': 235, 'e': 139, 't': 23, 'v': 25, 'j': 5, '\n': 46, 'E': 9, 's': 72, 'p': 16, 'r': 68, 'b': 33, 'i': 65, 'N': 1, 'ã': 8, 'g': 12, 'h': 10, 'A': 1, 'M': 10, 'c': 40, 'm': 68, 'q': 4, 'l': 35, 'f': 14, 'T': 3, 'ó': 8, ',': 3, 'z': 9, 'x': 5, 'á': 2, 'ê': 15, '?': 8, 'V': 14, 'L': 4, 'É': 4, '.': 18, 'S': 3}


Agora construimos nossa árvore com os caracteres, de forma que os nós mais próximos da raiz são aqueles que representam os caracteres com maior frequência. Isso fica especialmente fácil por termos utilizado uma heap, já guardando os nós em ordem de frequência. 

In [15]:
while(len(heap)>1):
    node1 = heapq.heappop(heap)
    node2 = heapq.heappop(heap)

    merged = HeapNode(None, node1.freq + node2.freq)
    merged.left = node1
    merged.right = node2

    heapq.heappush(heap, merged)

Estando com a árvore montada, podemos construir os códigos para cada caracter.

In [17]:

reverse_mapping = {}
codes = {}

def build_huffman_code(root, current_code):
    if(root == None):
        return

    if(root.char != None):
        codes[root.char] = current_code
        reverse_mapping[current_code] = root.char
        return

    build_huffman_code(root.left, current_code + "0")
    build_huffman_code(root.right, current_code + "1")
    
    
build_huffman_code(heapq.heappop(heap), "")
print(codes)

{'A': '000000', 'c': '000001', 'f': '000010', 'v': '000011', 'n': '00010', 'j': '00011', ' ': '001', 'o': '0100', 'i': '0101', 't': '0110', '\n': '0111', 'e': '1000', 's': '1001', 'm': '10100', 'b': '101010', 'u': '101011', 'g': '10110', 'r': '10111', 'd': '1100', 'l': '1101', 'a': '111'}


In [19]:
encoded_text = ""
for character in text:
    encoded_text += codes[character]
print(encoded_text)

00000000110100010110100010110111001111110110110101011101001110010000010100010110011110111011010001001011010000010110100010010110100000101111100010111110110010000010111110100110111111000111111100111011111001010111100100000111111011001111110111001111001110011101111111101100101101011101001010101111101000010010000111000001111100010101100100001011111010101000011101111000


In [20]:
extra_padding = 8 - len(encoded_text) % 8
for i in range(extra_padding):
    encoded_text += "0"

padded_info = "{0:08b}".format(extra_padding)
encoded_text = padded_info + encoded_text

In [27]:
compressed_path=f'{original_filename}_compressed.bin'
with open(compressed_path, 'wb') as output:

    btarray = bytearray()
    for i in range(0, len(encoded_text), 8):
        byte = encoded_text[i:i+8]
        btarray.append(int(byte, 2))        
        
    output.write(bytes(btarray))

In [32]:
original_file_size = os.path.getsize(original_path)
compressed_file_size = os.path.getsize(compressed_path)
size_reduction = format(round((((original_file_size-compressed_file_size)/original_file_size)*100), 0))
print(f'Nome do arquivo comprimido: {compressed_path}')
print(f'Tamanho do arquivo original: {original_file_size} bytes')
print(f'Tamanho do arquivo comprimido: {compressed_file_size} bytes')
print(f'O algoritmo conseguiu reduzir o tamanho do arquivo para {size_reduction}% do seu tamanho original')

Nome do arquivo comprimido: test_compressed.bin
Tamanho do arquivo original: 91 bytes
Tamanho do arquivo comprimido: 48 bytes
O algoritmo conseguiu reduzir o tamanho do arquivo para 47.0% do seu tamanho original


In [34]:
compressed_path = input("Digite o caminho do arquivo a ser descomprimido: ")
compressed_filename, compressed_file_extension = os.path.splitext(compressed_path)

with open(compressed_path, 'rb') as file:
    bits_string = ""

    byte = file.read(1)
    while(len(byte) > 0):
        byte = ord(byte)
        bits = bin(byte)[2:].rjust(8, '0')
        bits_string += bits
        byte = file.read(1)

Digite o caminho do arquivo a ser descomprimido: test_compressed.bin


In [36]:
padded_info = bits_string[:8]
extra_padding = int(padded_info, 2)

bits_string = bits_string[8:] 
encoded_text = bits_string[:-1*extra_padding]

In [37]:
current_code = ""
decompressed_text = ""

for bit in encoded_text:
    current_code += bit
    if(current_code in reverse_mapping):
        character = reverse_mapping[current_code]
        decompressed_text += character
        current_code = ""
print(decompressed_text)

A mimir alguma coisa
teste teste 
diagonal rajada
dude alsaldasda
alstrobalfo django 
livre


In [39]:
decompressed_file_path=f'{compressed_filename}_decompressed.txt'
with open(decompressed_file_path, 'w') as output:
    output.write(decompressed_text)
print(f'O arquivo descompactado foi salvo como {decompressed_file_path}')

O arquivo descompactado foi salvo como test_compressed_decompressed.txt


In [40]:
import filecmp
filecmp.cmp(original_path, decompressed_file_path)

True